In [1]:
import sys
print('Python version: ', sys.version)

Python version:  3.6.5 (default, May  3 2018, 09:34:46) 
[GCC 5.3.0]


In [12]:
from urllib import request
from io import StringIO
import csv
import os
import pandas as pd
from urllib.request import urlretrieve


In [14]:
isExists = os.path.exists("data")
isExists2 = os.path.exists("output")

if not isExists:
        os.makedirs("data")

if not isExists2:
        os.makedirs("output")

In [15]:
# Read Turnstile data from mta.info

datetime=200425

url='http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+str(datetime)+'.txt'

desfile="data/turnstile_"+str(datetime)+".txt"
print("Read from url:%s \r\ndesfile:%s" % (url,desfile))

urlretrieve(url, desfile)

Read from url:http://web.mta.info/developers/data/nyct/turnstile/turnstile_200425.txt 
desfile:data/turnstile_200425.txt


('data/turnstile_200425.txt', <http.client.HTTPMessage at 0x7fb8c24ad588>)

In [16]:
from pyspark import SparkContext 
from pyspark.sql import SQLContext
import pyspark.sql.functions as F


In [37]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('data/turnstile_200425.txt')
df.show()

+----+----+--------+-------+--------+--------+----------+--------+-------+-------+--------------------------------------------------------------------+
| C/A|UNIT|     SCP|STATION|LINENAME|DIVISION|      DATE|    TIME|   DESC|ENTRIES|EXITS                                                               |
+----+----+--------+-------+--------+--------+----------+--------+-------+-------+--------------------------------------------------------------------+
|A002|R051|02-00-00|  59 ST| NQR456W|     BMT|04/18/2020|00:00:00|REGULAR|7414753|                                                           2517634.0|
|A002|R051|02-00-00|  59 ST| NQR456W|     BMT|04/18/2020|04:00:00|REGULAR|7414754|                                                           2517635.0|
|A002|R051|02-00-00|  59 ST| NQR456W|     BMT|04/18/2020|08:00:00|REGULAR|7414756|                                                           2517643.0|
|A002|R051|02-00-00|  59 ST| NQR456W|     BMT|04/18/2020|12:00:00|REGULAR|7414765|      

In [38]:
# Clean data in turnstile.txt. Calculate the ridership for each station.

df  = df.withColumnRenamed("EXITS                                                               ", "EXITS")

df = df.withColumn('result', sum(df[col] for col in ['ENTRIES','EXITS']).cast('int'))
df = df.groupBy('STATION','DIVISION').sum('result').withColumnRenamed("sum(result)", "RIDESHIP (ENTRIES+EXITS)")

df = df.withColumn("STATION_low", F.lower(df.STATION))
df.show()

df.toPandas().to_csv('output/ridership.csv',mode='a', header=False)


+---------------+--------+------------------------+---------------+
|        STATION|DIVISION|RIDESHIP (ENTRIES+EXITS)|    STATION_low|
+---------------+--------+------------------------+---------------+
|BEDFORD PK BLVD|     IRT|               200125765|bedford pk blvd|
|    SARATOGA AV|     IRT|              5388787665|    saratoga av|
|    ATLANTIC AV|     BMT|               227760079|    atlantic av|
|   81 ST-MUSEUM|     IND|              3332713649|   81 st-museum|
|CONEY IS-STILLW|     BMT|             42117196805|coney is-stillw|
|  EAST BROADWAY|     IND|              8487077797|  east broadway|
| EXCHANGE PLACE|     PTH|               251225554| exchange place|
|         168 ST|     IRT|              1090784537|         168 st|
|      BERGEN ST|     IRT|              1253457672|      bergen st|
|          96 ST|     IND|              2519334106|          96 st|
| CATHEDRAL PKWY|     IRT|              1498803682| cathedral pkwy|
|          65 ST|     IND|              16502455

In [40]:
# Read data to get the location of stations

df2 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('data/Stations.csv')
df2.show()

+----------+----------+------------+--------+-------------------+--------------------+-------+--------------+---------+-------------+--------------+---------------------+---------------------+
|Station ID|Complex ID|GTFS Stop ID|Division|               Line|           Stop Name|Borough|Daytime Routes|Structure|GTFS Latitude|GTFS Longitude|North Direction Label|South Direction Label|
+----------+----------+------------+--------+-------------------+--------------------+-------+--------------+---------+-------------+--------------+---------------------+---------------------+
|         1|         1|         R01|     BMT|            Astoria|Astoria - Ditmars...|      Q|           N W| Elevated|    40.775036|    -73.912034|                 null|            Manhattan|
|         2|         2|         R03|     BMT|            Astoria|        Astoria Blvd|      Q|           N W| Elevated|    40.770258|    -73.917843|         Ditmars Blvd|            Manhattan|
|         3|         3|         R04

In [41]:
# Clean data in Station.csv. As each station may have several entrances, So calculate the average Latitude and Longitude for these 
# stations with several entrances.

df2 = df2.groupBy('Division','Stop Name').mean('GTFS Latitude','GTFS Longitude').withColumnRenamed("avg(GTFS Latitude)", "GTFS Latitude").withColumnRenamed("avg(GTFS Longitude)", "GTFS Longitude").withColumnRenamed("Stop Name", "STATION")

df2 = df2.withColumn("STATION_low", F.lower(df2.STATION))
df2.show()

df2.toPandas().to_csv('output/stop_location.csv')


+--------+--------------------+------------------+------------------+--------------------+
|Division|             STATION|     GTFS Latitude|    GTFS Longitude|         STATION_low|
+--------+--------------------+------------------+------------------+--------------------+
|     IRT|         New Lots Av|         40.666235|        -73.884079|         new lots av|
|     IRT|               28 St|        40.7451425|-73.98881449999999|               28 st|
|     IRT|Eastern Pkwy - Br...|         40.671987|        -73.964375|eastern pkwy - br...|
|     BMT|  Lexington Av/59 St|          40.76266|        -73.967258|  lexington av/59 st|
|     BMT|               18 Av|        40.6143125|-73.99607499999999|               18 av|
|     IND|                7 Av|40.714566500000004|-73.98097100000001|                7 av|
|     IND|           Ditmas Av|         40.636119|        -73.978172|           ditmas av|
|     IRT|         Winthrop St|         40.656652|          -73.9502|         winthrop st|

In [42]:
ridership_df = df
ridership_df.dtypes

[('STATION', 'string'),
 ('DIVISION', 'string'),
 ('RIDESHIP (ENTRIES+EXITS)', 'bigint'),
 ('STATION_low', 'string')]

In [43]:
location_df = df2
location_df.dtypes

[('Division', 'string'),
 ('STATION', 'string'),
 ('GTFS Latitude', 'double'),
 ('GTFS Longitude', 'double'),
 ('STATION_low', 'string')]

In [44]:
# Find the station names that are same in two lists.

first_left_join = df2.join(df, (df.DIVISION == df2.Division) & (df.STATION_low == df2.STATION_low),how='left')

first_left_join.show()
first_left_join.toPandas().to_csv('output/left_join.csv')


+--------+--------------------+------------------+------------------+--------------------+---------------+--------+------------------------+---------------+
|Division|             STATION|     GTFS Latitude|    GTFS Longitude|         STATION_low|        STATION|DIVISION|RIDESHIP (ENTRIES+EXITS)|    STATION_low|
+--------+--------------------+------------------+------------------+--------------------+---------------+--------+------------------------+---------------+
|     IRT|         New Lots Av|         40.666235|        -73.884079|         new lots av|    NEW LOTS AV|     IRT|              2814687477|    new lots av|
|     IRT|               28 St|        40.7451425|-73.98881449999999|               28 st|          28 ST|     IRT|            101973516545|          28 st|
|     IRT|Eastern Pkwy - Br...|         40.671987|        -73.964375|eastern pkwy - br...|           null|    null|                    null|           null|
|     BMT|  Lexington Av/59 St|          40.76266|        